In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np

import random
import sys
import io
import re

Using TensorFlow backend.
C:\Users\ICT01_11\.conda\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_11\.conda\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_11\.conda\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_11\.conda\envs\tf_test\lib

In [2]:
# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

path = '현진건-운수좋은날.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)

print('corpus length:', len(text))

corpus length: 10207


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 693


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 3389
Vectorization...


In [5]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...







In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/60
3389/3389 [==============================] - 28s 8ms/step - loss: 5.1940

----- Generating text after Epoch: 0
----- Generating with seed: "지 가잔 말을 들은 순간에 경련적으로 떠는 손, 유달리 큼직한 눈, 울 "
지 가잔 말을 들은 순간에 경련적으로 떠는 손, 유달리 큼직한 눈, 울 는              ,              .          지         고  는       지                ,                                     , ,                                        .       게              을        고  이        어                     지              지하       이더                                    의                     는 지                     를          .       ,   는   내        고             어               
Epoch 2/60
3389/3389 [==============================] - 27s 8ms/step - loss: 4.5798

----- Generating text after Epoch: 1
----- Generating with seed: "그는 병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다는 자기의 "
그는 병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다는 자기의       ”                  그      그     

3389/3389 [==============================] - 29s 9ms/step - loss: 3.1007

----- Generating text after Epoch: 12
----- Generating with seed: "한 때문이다. 그때도 김첨지가 오래간만에 돈을 얻어서 좁쌀 한 되와 십 "
한 때문이다. 그때도 김첨지가 오래간만에 돈을 얻어서 좁쌀 한 되와 십 전십 전에에 한 하를 소리에 어 에를 못에에 그 한리한는 그 전의지의 대에 하을 걸 에에 걸한 어내에 한 한 을을 대에 얼에에 눈에에 그삼에 오어에 그는 오찌에 한 에 들었에 기에에 한 은 내는 대에에마서 그 거에에 거어에의 대에 제 그 한 나에에 한에 마을 기생에 한 한 에리에 주에 기은에 한 오을 의 오기에 제에 기을 벌에에 들어, 그의에 에에 마었다. 그 그는 의에로 대에에 기에 의 손에 기찌에 하였다. 그 하의 어어운한 한에에 그 에 한 를 한 이의 기에에 거을 왜을 왜 울 지의 기에에에 어 어자을 대에에 김첨지의 대에에 어를 같을 웃어에 그 한 일시에 에에 비을 대 에 지는 기기에는 한 그의 내에 을 신져 추,에, 그 나의 의 를 그를 전에에 그 한 잔에에 집에 그의에게 하는 마기에게 오인한 내에에
Epoch 14/60
3389/3389 [==============================] - 29s 8ms/step - loss: 2.9587

----- Generating text after Epoch: 13
----- Generating with seed: " 눌러 곱빼기 한 잔을 또 마셨다. 김첨지의 눈은 벌써 개개 풀리기 시작"
 눌러 곱빼기 한 잔을 또 마셨다. 김첨지의 눈은 벌써 개개 풀리기 시작 이 운다. 또 대이 그을 듯이 도 지의 듯이 도 왔은 가이 그 지이 말이 , 지이 같은 .을 내가 말이 비려 비을 이이 , 붙이  이 비다. 그 나을 말이 이 질가 의 나 있를 못 다. 그 기고 추은 이 말다. 이 하고 울무며 시이 . 못이 나은 나,

, 빈대떡…….이 너저분하게 늘어 놓은 안주 탁자에 김첨지는 갑자기 속이 쓰은 이 그 없을 그 그의 그 김첨지의 채 치이 사을 얼 이 한 떨의 벌대이 돈까 이  깔을 길 그의 내리에 넘 더 치 한 어어 제 이 한양적그 그의 물을 이 그의도 그 치기 가기 이  더을 바 었다. 불로 비가 잉  비어 얼서 꿈  마한 내리는 이 떨기로 생 치나 그기 우도 정 진를 내리가 있고 김첨지의 코리 김첨지는 시화 이 김첨지게 대한 마속에 어어오며  한 전을 정작 김첨지는 눈에 벌하였다. “여 이 같십 대은 이 터의 맞판 적 인리 내어!” 하고 김첨지의 눈금 불어터 거 다. 치 이 비면 치기를 돈 그 을 할 수 그 한 없의 그 연제 남기 이 그의 마에 비려 더  많서 마비 치하였다. 그날 그러다. 오늘은 얼이 그 언라이 그 그 소리가 기 를 없이 전을 정이 그 그 떨어에 들어오어어 제 같이 , 
Epoch 26/60
3389/3389 [==============================] - 26s 8ms/step - loss: 0.2137

----- Generating text after Epoch: 25
----- Generating with seed: "섞인 추기가 무딘 김첨지의 코를 찔렀다. 방안에 들어서며 설렁탕을 한구석"
섞인 추기가 무딘 김첨지의 코를 찔렀다. 방안에 들어서며 설렁탕을 한구석그리는 이 벌었일가에 그 나, 구 이 벌를 맞아 서 있시. 그 그의 그이 우서 길 지의 맞에 울 안 돈시할 한 그 없을 남가 지는 더이 그 그의 걸은 치이 눈을 얼거거? 는 이시도 한 그 그리 청달 이 무시한 그 그 나릇 막 전 한 모마를 병이 보다. 이 그 잔운에 들 떨어. 기 전 없운다. 그 오일 년이 전 터 일 인 한 전 한 그 그리에 그 그의 물이 그 그의 그리에 없 한 잔를 전에 정려. 뒤 그 한 신의 전에 오기질 무서 길 지 있운 운을 내어?” 김첨지는 눈한 벌를 개여러개  이어. 그 하고 전 하고 손자을 중 에 일 못 잔 전운라 무온 병 무 한 그 신양은 그 그